This is one of the Objectiv [example notebooks](https://objectiv.io/docs/modeling/example-notebooks/). These notebooks can also run [on your own data](https://objectiv.io/docs/modeling/get-started-in-your-notebook/) (see [how to set up tracking](https://objectiv.io/docs/tracking/)).

# Open Model Hub basics
This example notebook demonstrates how you can use the pre-built models from the [open model hub](https://objectiv.io/docs/modeling/) in conjunction with modeling library [Bach](https://objectiv.io/docs/modeling/bach/) to quickly build model stacks to answer common analytics questions.

For more background and an overview of all available models, see the [open model hub documentation](https://objectiv.io/docs/modeling/open-model-hub/).

## Using the Open Model Hub
The following types of functions/models are provided:

1. [Helper functions](https://objectiv.io/docs/modeling/open-model-hub/models/helper-functions/): Simplify manipulating and analyzing the data.
2. [Aggregation models](https://objectiv.io/docs/modeling/open-model-hub/models/aggregation/): Enable running some of the more common data analyses and product analytics metrics.
3. [Machine learning models](https://objectiv.io/docs/modeling/open-model-hub/models/machine-learning/): ML models such as logistic regression.
4. [Funnels](https://objectiv.io/docs/modeling/open-model-hub/models/funnels/): To analyze Funnels, e.g. discover all the (top) user journeys that lead to conversion or drop-off.

Modeling behavior of users and groups is enabled through configurable [Identity Resolution](https://objectiv.io/docs/modeling/open-model-hub/identity-resolution/).

**Helper functions** always return a [Series](https://objectiv.io/docs/modeling/bach/api-reference/Series/) with the same shape and index as the [DataFrame](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/) they are applied to. This ensures they can be added as a column to that [DataFrame](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/). Helper functions that return [SeriesBoolean](https://objectiv.io/docs/modeling/bach/api-reference/Series/Boolean/) can be used to filter the data. The helper functions can be accessed with the `map` accessor from a model hub instance.

**Aggregation models** perform multiple [Bach](https://objectiv.io/docs/modeling/bach/) instructions that run some of the more common data analyses and product analytics metrics. They always return aggregated data in some form from the [DataFrame](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/) they're applied to. Aggregation models can be accessed with the `aggregate` accessor from a model hub instance.

Most of the model hub helper functions and aggregation models take `data` as their first argument: this is
the [DataFrame](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/) with the Objectiv data to apply the model to. 

For an example of a **machine learning model**, see the [logistic regression example notebook](https://objectiv.io/docs/modeling/example-notebooks/logistic-regression/).

Below we'll showcase a selection of the models from the open model hub.

## Get started
We first have to instantiate the model hub and an Objectiv DataFrame object.

In [ ]:
# set the timeframe of the analysis
start_date = '2022-03-01'
end_date = None

In [ ]:
from modelhub import ModelHub, display_sql_as_markdown
from sql_models.util import is_athena

# instantiate the model hub and set the default time aggregation to daily
# and set the global contexts that will be used in this example
modelhub = ModelHub(time_aggregation='%Y-%m-%d', global_contexts=['application'])
# get a Bach DataFrame with Objectiv data within a defined timeframe
df = modelhub.get_objectiv_dataframe(start_date=start_date, end_date=end_date)

### Reference
* [modelhub.ModelHub](https://objectiv.io/docs/modeling/open-model-hub/api-reference/ModelHub/ModelHub/)
* [modelhub.ModelHub.get_objectiv_dataframe](https://objectiv.io/docs/modeling/open-model-hub/api-reference/ModelHub/get_objectiv_dataframe/)

## Run a simple aggregation model
Calculating the number of unique users is one of the models. As it is an aggregation model, it's called with [`model_hub.aggregate.unique_users()`](https://objectiv.io/docs/modeling/open-model-hub/models/aggregation/unique_users/) (or `model_hub.agg.unique_users()` for short). It uses the `time_aggregation` that is set when the model hub was instantiated. 

With [`.head()`](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/head/) we immediately query the data to show the results. [`.to_pandas()`](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/to_pandas/) can be used to use all results as a pandas object in Python. These (and the following) results are sorted descending, so we show the latest data first.

In [ ]:
# calculate the unique users per set time_aggregation (in this case per day)
users = modelhub.aggregate.unique_users(df)
users.sort_index(ascending=False).head(10)

### Reference
* [modelhub.Aggregate.unique_users](https://objectiv.io/docs/modeling/open-model-hub/models/aggregation/unique_users/)
* [bach.DataFrame.sort_index](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/sort_index/)
* [bach.DataFrame.head](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/head/)

## Mapping and filtering
Here we'll use `map` operations from the open model hub to label certain users or events, in order to filter on them or combine them with other models later on.

First, we'll label users as being a new user. As `time_aggregation` was set to '%Y-%m-%d' it means all hits are labeled as new for the entire day in which the user had its first session.

In [ ]:
# label new users & all their events on the defined time_aggregation 
df['is_new_user'] = modelhub.map.is_new_user(df)
df.is_new_user.head(10)

We can also label conversion events. To do this, we first have to define what a conversion is, by setting the type of event and the location in the product where the event was triggered ([see more about the location stack here](https://objectiv.io/docs/modeling/example-notebooks/open-taxonomy/#location-stack-and-global-contexts)), using [`add_conversion_event`](https://objectiv.io/docs/modeling/open-model-hub/api-reference/ModelHub/add_conversion_event/).

In [ ]:
# define a conversion event, and label events whether they converted
modelhub.add_conversion_event(location_stack=df.location_stack.json[{'id': 'Quickstart Guide', '_type': 'LinkContext'}:],
                              event_type='PressEvent',
                              name='quickstart_presses')
df['conversion_events'] = modelhub.map.is_conversion_event(df, 'quickstart_presses')
df.conversion_events.head(10)

### Combine mapping with filtering and aggregation
As the map functions above return a SeriesBoolean, they can be combined with filter and aggregation models. We use the same aggregation model we showed earlier ([`unique_users`](https://objectiv.io/docs/modeling/open-model-hub/models/aggregation/unique_users/)), but now applying the `df.conversion_events` filter to just look at unique converted users per day.

In [ ]:
# filter unique users on whether they've converted
modelhub.aggregate.unique_users(df[df.conversion_events]).sort_index(ascending=False).head(10)

Other aggregation models can be used similarly. In the example below, the average session duration is calculated just for new users.

In [ ]:
# calculate average session duration, filtered on new users
duration_new_users = modelhub.aggregate.session_duration(df[df.is_new_user])
duration_new_users.sort_index(ascending=False).head(10)

### Using multiple model hub filters

The model hub's `map` results can be combined and reused. Below we set the result of two helper functions as a column to the original DataFrame, and use them both to filter the data and apply an aggregation model. We calculate the number of users that were new in a month, and also the number of users that converted twice on a day.

In [ ]:
# calculate new users & users that converted twice on a day
df['is_new_user_month'] = modelhub.map.is_new_user(df, time_aggregation = '%Y-%m')
df['is_twice_converted'] = modelhub.map.conversions_in_time(df, name='quickstart_presses')==2
# use results in an aggregation model
modelhub.aggregate.unique_users(df[df.is_new_user_month & df.is_twice_converted]).sort_index(ascending=False).head()

### Reference
* [modelhub.Map.is_new_user](https://objectiv.io/docs/modeling/open-model-hub/models/helper-functions/is_new_user/)
* [bach.DataFrame.head](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/head/)
* [modelhub.ModelHub.add_conversion_event](https://objectiv.io/docs/modeling/open-model-hub/api-reference/ModelHub/add_conversion_event/)
* [modelhub.Map.is_conversion_event](https://objectiv.io/docs/modeling/open-model-hub/models/helper-functions/is_conversion_event/)
* [modelhub.Aggregate.unique_users](https://objectiv.io/docs/modeling/open-model-hub/models/aggregation/unique_users/)
* [modelhub.Aggregate.session_duration](https://objectiv.io/docs/modeling/open-model-hub/models/aggregation/session_duration/)
* [modelhub.Map.conversions_in_time](https://objectiv.io/docs/modeling/open-model-hub/models/helper-functions/conversions_in_time/)
* [bach.DataFrame.sort_index](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/sort_index/)

## Combine model results
Results from aggregation models can be used together if they share the same index type - similar to pandas. Below the share of new users per day is calculated.

In [ ]:
# calculate the share of new users per day using results from two aggregation models
new_user_share = modelhub.agg.unique_users(df[df.is_new_user]) / modelhub.agg.unique_users(df)
new_user_share.sort_index(ascending=False).head(10)

### Reference
* [modelhub.Aggregate.unique_users](https://objectiv.io/docs/modeling/open-model-hub/models/aggregation/unique_users/)
* [modelhub.Map.is_new_user](https://objectiv.io/docs/modeling/open-model-hub/models/helper-functions/is_new_user/)
* [bach.DataFrame.sort_index](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/sort_index/)
* [bach.DataFrame.head](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/head/)

## Crunch data further with the Bach modeling library
All results from the model hub are in the form of a Bach [DataFrame](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/) or [Series](https://objectiv.io/docs/modeling/bach/api-reference/Series/). This makes the open model hub and Bach work seamlessly together.

In [ ]:
# We'll do a lot of operations on the data in the df DataFrame. To make this easier for the
# database (especially BigQuery), we tell Bach to materialize the current DataFrame as temporary
# table. This statement has no direct effect, but any invocation of head() on the DataFrame later
# on will consist of two queries: one to create a temporary table with the current state of the
# DataFrame, and one that queries that table and does subsequent operations.
# N.B. Athena doesn't support temporary tables.
if not is_athena(df.engine):
    df = df.materialize(materialization='temp_table')

In [ ]:
# now label the number of times a user is converted in a session at a certain moment
df['conversion_count'] = modelhub.map.conversions_in_time(df, name='quickstart_presses')

# use Bach to do any supported operation using pandas syntax.
# select users that converted
converted_users = df[df.conversion_events].user_id.unique()
# select PressEvents of users that converted
df_selection = df[(df.event_type == 'PressEvent') &
                  (df.user_id.isin(converted_users))]
# calculate the number of PressEvents before conversion per session
presses_per_session = df_selection[df_selection.conversion_count == 0].groupby('session_id').session_hit_number.count()
presses_per_session_pdf = presses_per_session.to_pandas()

Now let's see the results, at which point the underlying query is actually executed.

In [ ]:
presses_per_session_pdf.head()

See the [Bach API reference](https://objectiv.io/docs/modeling/bach/api-reference/) for all available operations.

### Reference
* [bach.DataFrame.materialize](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/materialize/)
* [modelhub.Map.conversions_in_time](https://objectiv.io/docs/modeling/open-model-hub/models/helper-functions/conversions_in_time/)
* [bach.Series.unique](https://objectiv.io/docs/modeling/bach/api-reference/Series/unique/)
* [bach.Series.isin](https://objectiv.io/docs/modeling/bach/api-reference/Series/isin/)
* [bach.DataFrame.groupby](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/groupby/)
* [bach.DataFrame.head](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/head/)

## Export results to a pandas DataFrame
Bach DataFrames and/or model hub results can always be exported to pandas, so you can use all its options as well as pandas-compatible ML packages. Since Bach DataFrame operations run on the full dataset in the SQL data store, it is recommended to export to pandas if the data is small enough, i.e. by aggregation or selection.

Below we plot the previously calculated presses per session before conversion, using pandas' built-in plotting methods.

In [ ]:
presses_per_session_pdf.hist()

### Reference
* [bach.DataFrame.to_pandas](https://objectiv.io/docs/modeling/bach/api-reference/DataFrame/to_pandas/)

## Get the SQL for any analysis
The SQL for any analysis can be exported with one command, so you can use models in production directly to simplify data debugging & delivery to BI tools like Metabase, dbt, etc. See how you can [quickly create BI dashboards with this](https://objectiv.io/docs/home/up#creating-bi-dashboards).

In [ ]:
# show SQL for analysis; this is just one example, and works for any Objectiv model/analysis
# complex SQL statement alert!
display_sql_as_markdown(new_user_share)

That's it! [Join us on Slack](https://objectiv.io/join-slack) if you have any questions or suggestions.

# Next Steps

### Use this notebook with your own data

You can use the example notebooks on any dataset that was collected with Objectiv's tracker, so feel free to 
use them to bootstrap your own projects. They are available as Jupyter notebooks on our [GitHub repository](https://github.com/objectiv/objectiv-analytics/tree/main/notebooks). See [instructions to set up the Objectiv tracker](https://objectiv.io/docs/tracking/).

### Check out related example notebooks

* [Open taxonomy how-to](https://objectiv.io/docs/modeling/example-notebooks/open-taxonomy/) - see what you can do with the Bach modeling library and a dataset that is validated against the open analytics taxonomy.

* [Explore your data](https://objectiv.io/docs/modeling/example-notebooks/explore-data/) - see how you can easily explore your data collected with Objectiv.